#  COVID 19 #

In [1]:
# Importazione librerie operative
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [2]:
# Lettura dati sperimentali via web
DF = pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv')

In [3]:
@interact(parametro=DF.columns[2:16])
def sel(parametro):
    fg=go.Figure()
    return fg.add_trace(go.Scatter(x=DF.data ,y=DF[parametro], mode='markers', marker_color = 'green'))

interactive(children=(Dropdown(description='parametro', options=('ricoverati_con_sintomi', 'terapia_intensiva'…

In [4]:
DF['data'] = DF['data'].astype('datetime64[ns]')
DF['data'] = DF['data'].dt.strftime('%d/%m')

In [5]:
tab = DF.copy(deep=True)
tab = tab.drop(['stato','note','casi_da_sospetto_diagnostico','casi_da_screening',
            'tamponi','casi_testati','totale_ospedalizzati','totale_positivi','totale_casi',
               'variazione_totale_positivi'],axis=1)
tab = tab.set_index('data')
tab

,ricoverati_con_sintomi,terapia_intensiva,isolamento_domiciliare,nuovi_positivi,dimessi_guariti,deceduti
data,,,,,,
24/02,101,26,94,221,1,7
25/02,114,35,162,93,1,10
26/02,128,36,221,78,3,12
27/02,248,56,284,250,45,17
28/02,345,64,412,238,46,21
...,...,...,...,...,...,...
19/08,866,66,14428,642,204506,35412
20/08,883,68,15063,845,204686,35418
21/08,919,69,15690,947,204960,35427


In [6]:
df = tab.copy(deep=True)
# PCA
pca = PCA(n_components=2)
# full data:
xs = scale(df)  
x_red = pca.fit_transform(xs)
V = pca.components_
nPC = np.arange(len(V)) + 1
dfV = pd.DataFrame(V, columns=[df.columns], index=[nPC])

In [7]:
# Loadings plot
arr = np.array(dfV)
label = np.array(df.columns)
xtit = 'PC1 - explained variance = '+str(np.round(100*pca.explained_variance_ratio_[0],decimals=2))+'%'
ytit = 'PC2 - explained variance = '+str(np.round(100*pca.explained_variance_ratio_[1],decimals=2))+'%'
plot = go.Scatter(x=arr[0,:], y=arr[1,:], mode='markers+text',
                  marker=dict(symbol=[201],color='darkred', size=10), 
              text = label, textposition='middle left')
fig=go.Figure()
fig.add_trace(plot)
fig.update_layout(height=600, width=800, title='PCA - Loadings plot - from 24th of February',
                  xaxis_title=xtit,yaxis_title=ytit, 
                  xaxis_zeroline=True, yaxis_zeroline=True,
                  xaxis_zerolinecolor='blue', yaxis_zerolinecolor='blue')
fig.update_xaxes(range=[min(arr[0,:])-0.2, max(arr[0,:])+0.2])
fig.update_yaxes(range=[min(arr[1,:])-0.2, max(arr[1,:])+0.2])

In [8]:
# Score plot
col = np.arange(len(df.index))
fig=go.Figure()
fig.add_trace(go.Scatter(x=x_red[:,0], y=x_red[:,1], mode='markers+lines', marker=dict(symbol=[200], 
             color = col, line_width=2, size=10), hovertext=(df.index)))
fig.update_layout(height=600, width=800, title='PCA - Score plot - from 24th of February', 
                  xaxis_title=xtit,yaxis_title=ytit, showlegend=False,
                  xaxis_zeroline=True, yaxis_zeroline=True, 
                  xaxis_zerolinecolor='blue', yaxis_zerolinecolor='blue')

In [9]:
df = tab.copy(deep=True)
# PCA
pca = PCA(n_components=2)
# last two months:
xs = scale(df.tail(60))
x_red = pca.fit_transform(xs)
V = pca.components_
nPC = np.arange(len(V)) + 1
dfV = pd.DataFrame(V, columns=[df.columns], index=[nPC])

In [10]:
# Loadings plot
arr = np.array(dfV)
label = np.array(df.columns)
xtit = 'PC1 - explained variance = '+str(np.round(100*pca.explained_variance_ratio_[0],decimals=2))+'%'
ytit = 'PC2 - explained variance = '+str(np.round(100*pca.explained_variance_ratio_[1],decimals=2))+'%'
plot = go.Scatter(x=arr[0,:], y=arr[1,:], mode='markers+text',
                  marker=dict(symbol=[201],color='darkred', size=10), 
              text = label, textposition='middle left')
fig=go.Figure()
fig.add_trace(plot)
fig.update_layout(height=600, width=800, title='PCA - Loadings plot - last 60dd', xaxis_title=xtit,yaxis_title=ytit, 
                  xaxis_zeroline=True, yaxis_zeroline=True,
                  xaxis_zerolinecolor='blue', yaxis_zerolinecolor='blue')
fig.update_xaxes(range=[min(arr[0,:])-0.2, max(arr[0,:])+0.2])
fig.update_yaxes(range=[min(arr[1,:])-0.2, max(arr[1,:])+0.2])

In [11]:
# Score plot
col = np.arange(len(df.index))
fig=go.Figure()
fig.add_trace(go.Scatter(x=x_red[:,0], y=x_red[:,1], mode='markers+lines', marker=dict(symbol=[200], 
             color = col, line_width=2, size=10), hovertext=(DF.data.tail(60))))
fig.update_layout(height=600, width=800, title='PCA - Score plot - last 60dd', 
                  xaxis_title=xtit,yaxis_title=ytit, showlegend=False,
                  xaxis_zeroline=True, yaxis_zeroline=True, 
                  xaxis_zerolinecolor='blue', yaxis_zerolinecolor='blue')